In [1]:
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input
from keras.models import Model
import os
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import random

2022-12-08 19:03:10.419095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.run_functions_eagerly(True)
def set_seeds(seed=1299827):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

## 1. encode user info

In [3]:
names = ['occupations']
data_dir = "./data/ml-100k"
occupations = pd.read_csv(os.path.join(data_dir, 'u.occupation'), '\t', names=names,
                       engine='python')

/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
names = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
data_dir = "./data/ml-100k"
user_info = pd.read_csv(os.path.join(data_dir, 'u.user'), '|', names=names,
                       engine='python')



/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
        ds = ds.prefetch(batch_size)
    return ds

def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [6]:
user_info.drop(columns=['user_id'], inplace=True)
gender = {'M': 0, 'F': 1}
user_info['gender'] = user_info['gender'].apply(lambda x: gender[x])

In [7]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
        values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [8]:
binary_feature_names = ['gender']
categorical_feature_names = ['occupation', 'zip_code']
numeric_features = user_info[['age']]

inputs = {}
for name, column in user_info.items():
#     if type(column[0]) == str:
#         dtype = tf.string
    if (name in categorical_feature_names or
        name in binary_feature_names):
        dtype = tf.int64
    else:
        dtype = tf.float32

    inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)


In [9]:
preprocessed = []

for name in binary_feature_names:
    inp = inputs[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed.append(float_value)

2022-12-08 19:03:19.052533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [11]:
users = tf.convert_to_tensor(user_info['gender'].values, dtype=tf.float32)

In [12]:
users = tf.concat((users[:, tf.newaxis], tf.squeeze(normalizer(user_info['age'].values))[:, tf.newaxis]), axis=1)

In [13]:
numeric_inputs = {}
for name in ['age']:
    numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

In [14]:
for name in categorical_feature_names:
    vocab = sorted(set(user_info[name]))
    print(f'name: {name}')
#     print(f'vocab: {vocab}\n')

    if type(vocab[0]) is str:
        lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot', num_oov_indices=0)
    else:
        lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot', num_oov_indices=0)

    x = lookup(user_info[name].values)
    users = tf.concat((users, x), axis=1)

name: occupation
name: zip_code


/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/numpy/core/numeric.py:2446: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [15]:
encoding_dim = 32
user_input = keras.Input(shape=(818,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(128, activation='relu')(user_input)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(818)(decoded)

user_autoencoder = keras.Model(user_input, decoded)
user_autoencoder.compile(optimizer='adam', loss='mse')

user_encoder = keras.Model(user_input, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_output = user_autoencoder.layers[-3](encoded_input)
decoder_output = user_autoencoder.layers[-2](decoder_output)
decoder_output = user_autoencoder.layers[-1](decoder_output)

# Create the decoder model
user_decoder = keras.Model(encoded_input, decoder_output)

In [16]:
user_autoencoder.compile(optimizer='adam', loss='mse')

In [17]:
set_seeds()
user_autoencoder.fit(users, users,
                epochs=100,
                batch_size=256,
                shuffle=True)

Epoch 1/100
4/4 [==============================] - 0s 47ms/step - loss: 0.0040
Epoch 2/100
4/4 [==============================] - 0s 58ms/step - loss: 0.0040
Epoch 3/100
4/4 [==============================] - 0s 49ms/step - loss: 0.0040
Epoch 4/100
4/4 [==============================] - 0s 48ms/step - loss: 0.0039
Epoch 5/100
4/4 [==============================] - 0s 50ms/step - loss: 0.0039
Epoch 6/100
4/4 [==============================] - 0s 49ms/step - loss: 0.0038
Epoch 7/100
4/4 [==============================] - 0s 48ms/step - loss: 0.0037
Epoch 8/100
4/4 [==============================] - 0s 51ms/step - loss: 0.0035
Epoch 9/100
4/4 [==============================] - 0s 47ms/step - loss: 0.0033
Epoch 10/100
4/4 [==============================] - 0s 44ms/step - loss: 0.0030
Epoch 11/100
4/4 [==============================] - 0s 46ms/step - loss: 0.0028
Epoch 12/100
4/4 [==============================] - 0s 45ms/step - loss: 0.0027
Epoch 13/100
4/4 [==============================]

In [22]:
latent_user = user_encoder.predict(users)

30/30 [==============================] - 0s 3ms/step


## 2. encode item info

In [16]:
from transformers import AutoTokenizer
from transformers import TFAutoModel
from keras.optimizers import Adam

/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [18]:
names = ['movie_id', 'movie_title', 'release_date', 'video_release_date',
              'IMDb_URL', 'unknown', 'Action', 'Adventure' 'Animation',
              "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
              "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
              "Thriller", "War", "Western"]
data_dir = "./data/ml-100k"
item_info = pd.read_csv(os.path.join(data_dir, 'u.item'), '|', names=names,
                       engine='python', encoding = "ISO-8859-1", index_col=False)

/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/pp/m070wqfs1jl8k6stqvty0wkc0000gn/T/ipykernel_19974/74995158.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  item_info = pd.read_csv(os.path.join(data_dir, 'u.item'), '|', names=names,


In [19]:
item_info.drop(columns=['movie_id', 'video_release_date', 'IMDb_URL'], inplace=True)

In [20]:
item_info['movie_title'] = item_info['movie_title'].str.replace(r" \(\d{4}\)","")

/var/folders/pp/m070wqfs1jl8k6stqvty0wkc0000gn/T/ipykernel_19974/1251032369.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  item_info['movie_title'] = item_info['movie_title'].str.replace(r" \(\d{4}\)","")


In [21]:
lm_input = tokenizer(item_info['movie_title'].values.tolist(), return_tensors='tf', truncation=True, padding=True, max_length=8)

In [22]:
model = TFAutoModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [23]:
lm_output = model.predict(lm_input)

/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


53/53 [==============================] - 32s 598ms/step


In [24]:
items = tf.reshape(lm_output[0], (1682, -1))

In [25]:
item_info['release_year'] = item_info['release_date'].str.extract(r'(\d{4}$)')
item_info['release_year'] = item_info['release_year'].apply(lambda x: int(x) if pd.notnull(x) else x)

In [26]:
item_info['day_sin'] = np.sin(2 * np.pi * (pd.to_datetime(item_info['release_date']).dt.dayofyear/365.0))
item_info['day_cos'] = np.cos(2 * np.pi * (pd.to_datetime(item_info['release_date']).dt.dayofyear/365.0))

In [27]:
item_info.drop(columns=['release_date'], inplace=True)

In [28]:
item_info.fillna(item_info.mean(), inplace=True)

/var/folders/pp/m070wqfs1jl8k6stqvty0wkc0000gn/T/ipykernel_19974/1321115318.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  item_info.fillna(item_info.mean(), inplace=True)


In [29]:
items = tf.concat((items, tf.convert_to_tensor(item_info.iloc[:, 1:19], dtype=tf.float32)),axis=1)

In [30]:
numeric_features = item_info.columns.tolist()

inputs = {}
for name, column in item_info.items():
#     if type(column[0]) == str:
#         dtype = tf.string
    if (name in categorical_feature_names or
        name in binary_feature_names):
        dtype = tf.int64
    else:
        dtype = tf.float32

    inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

In [31]:
preprocessed = []

for name in numeric_features[:-3]:
    inp = inputs[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed.append(float_value)

In [32]:
numeric_inputs = {}
for name in ['release_year']:
    numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

In [33]:
items = tf.concat((items, tf.squeeze(normalizer(item_info['release_year'].values))[:,tf.newaxis]), axis=1)

In [34]:
items = tf.concat((items, tf.convert_to_tensor(item_info.iloc[:,-2:], dtype=tf.float32)), axis=1)

In [35]:
for name in numeric_features[-2:]:
    inp = inputs[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed.append(float_value)

In [36]:
encoding_dim = 32
item_input = keras.Input(shape=(6166,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(1024, activation='relu')(item_input)
encoded = layers.Dense(128, activation='relu')(encoded)
encoded = layers.LayerNormalization()(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)
encoded = layers.LayerNormalization()(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(128, activation='relu')(encoded)
decoded = layers.Dense(1024, activation='relu')(decoded)
decoded = layers.Dense(6166)(decoded)


item_autoencoder = keras.Model(item_input, decoded)
item_autoencoder.compile(optimizer='adam', loss='mse')

item_encoder = keras.Model(item_input, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_output = item_autoencoder.layers[-3](encoded_input)
decoder_output = item_autoencoder.layers[-2](decoder_output)
decoder_output = item_autoencoder.layers[-1](decoder_output)

# Create the decoder model
item_decoder = keras.Model(encoded_input, decoder_output)

In [39]:
item_autoencoder.compile(optimizer='adam', loss='mse')

In [40]:
set_seeds()
item_autoencoder.fit(items, items,
                epochs=100,
                batch_size=256,
                shuffle=True)

Epoch 1/100


/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


7/7 [==============================] - 3s 296ms/step - loss: 4.3155
Epoch 2/100
7/7 [==============================] - 2s 284ms/step - loss: 4.0910
Epoch 3/100
7/7 [==============================] - 2s 279ms/step - loss: 3.6608
Epoch 4/100
7/7 [==============================] - 2s 275ms/step - loss: 3.0031
Epoch 5/100
7/7 [==============================] - 2s 277ms/step - loss: 2.0465
Epoch 6/100
7/7 [==============================] - 2s 324ms/step - loss: 0.7826
Epoch 7/100
7/7 [==============================] - 2s 289ms/step - loss: 0.1703
Epoch 8/100
7/7 [==============================] - 2s 275ms/step - loss: 0.1156
Epoch 9/100
7/7 [==============================] - 2s 274ms/step - loss: 0.0591
Epoch 10/100
7/7 [==============================] - 2s 289ms/step - loss: 0.0420
Epoch 11/100
7/7 [==============================] - 2s 272ms/step - loss: 0.0361
Epoch 12/100
7/7 [==============================] - 2s 272ms/step - loss: 0.0339
Epoch 13/100
7/7 [==============================]

In [41]:
latent_item = item_encoder.predict(items)

53/53 [==============================] - 1s 12ms/step


In [42]:
item_decoder.predict(latent_item)[0][0]

53/53 [==============================] - 0s 8ms/step


-0.010121994

## User-Item match dataset

In [37]:
user_ids = []
item_ids = []
ratings = []
timestamps = []

with open("./data/ml-100k/u.data", 'rt') as file1:
    for line in file1.readlines():
        a = line.split()
        user_ids.append(a[0])
        item_ids.append(a[1])
        ratings.append(a[2])
        timestamps.append(a[3])
    
rating_df = pd.DataFrame({'user_id': user_ids, 'item_id': item_ids, 'rating': ratings, 'timestamp': timestamps})
rating_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [38]:
# tf.map_fn(tf.one_hot, tf.convert_to_tensor(rating_df['rating'].values))

In [39]:
sigmoid = lambda x: 1/(1 + np.exp(-x))

In [40]:
rating_similarity = {1: sigmoid(-4.), 2: sigmoid(-2.), 3: sigmoid(0.), 4: sigmoid(2.), 5: sigmoid(4.)}

In [41]:
rating_df['rating']=rating_df['rating'].apply(lambda x: rating_similarity[int(x)])

In [42]:
rating_df.drop(columns=['timestamp'], inplace=True)

In [43]:
rating_df['user_id'] = rating_df['user_id'].astype(int)-1
rating_df['item_id'] = rating_df['item_id'].astype(int)-1

In [44]:
user = tf.gather(users, rating_df.user_id.values)
item = tf.gather(items, rating_df.item_id.values)

In [45]:
# dataset = tf.data.Dataset.from_tensor_slices((user, item, tf.convert_to_tensor(rating_df['rating'].values)))

In [46]:
user_input = Input(shape=(818,))
item_input = Input(shape=(6166,))
user_feat = user_encoder(user_input)
item_feat = item_encoder(item_input)
outputs = tf.keras.layers.Dot(axes=1)([user_feat, item_feat])
# outputs =  tf.keras.activations.sigmoid(dot_similarity)

In [47]:
model = Model(inputs=[user_input, item_input], outputs=outputs)

In [48]:
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer="adam",
	metrics=["accuracy"])

In [ ]:
set_seeds()
history = model.fit(
    [user, item], tf.convert_to_tensor(rating_df['rating'].values),
    batch_size=128, 
    epochs=10, shuffle=True)

Epoch 1/10


/Users/jamee/miniconda3/envs/Transformer/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


782/782 [==============================] - 144s 184ms/step - loss: 0.6021 - accuracy: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 168s 215ms/step - loss: 0.5774 - accuracy: 0.0000e+00
Epoch 3/10
782/782 [==============================] - 167s 214ms/step - loss: 0.5710 - accuracy: 0.0000e+00
Epoch 4/10
782/782 [==============================] - 165s 211ms/step - loss: 0.5655 - accuracy: 0.0000e+00
Epoch 5/10
782/782 [==============================] - 187s 239ms/step - loss: 0.5601 - accuracy: 0.0000e+00
Epoch 6/10
102/782 [==>...........................] - ETA: 3:12 - loss: 0.5521 - accuracy: 0.0000e+00

In [57]:
tf.math.sigmoid(model.predict([user[:8], item[:8]]))

1/1 [==============================] - 0s 114ms/step


<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.81873375],
       [0.7829724 ],
       [0.11172543],
       [0.69215775],
       [0.5426289 ],
       [0.836669  ],
       [0.5364756 ],
       [0.8023394 ]], dtype=float32)>

In [63]:
rating_df['rating'][:8]

0    0.500000
1    0.500000
2    0.017986
3    0.119203
4    0.017986
5    0.880797
6    0.119203
7    0.982014
Name: rating, dtype: float64